# Objective
Build a system that allows any company employee to consult, in natural language, information from the internal knowledge base (e.g., how the company's main model works, details about motorcycle couriers and services).

# Importing Libraries

In [ ]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from IPython.display import Markdown, display
import shutil
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory,ChatMessageHistory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
import logging

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

In [ ]:
from pydantic import BaseModel, Field
from datetime import datetime
import json
from pathlib import Path
from threading import Thread
import json

# Loading Knowlege Base

In [ ]:
# Function to add metadata
def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

# Adjusting utf-8 
text_loader_kwargs = {'encoding': 'utf-8'}

# Path to folder with the files
folders = glob.glob("knowledge-base/*")

# Reading files
documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

In [ ]:
# Creating chunks
text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=160)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

# Vector Storage


In [ ]:
# Embeddings open source
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

In [ ]:
db_name = "vector_db"

In [ ]:
# Delete if already exists
if os.path.exists(db_name):
    shutil.rmtree(db_name)
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [ ]:
# Create vectorstore
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

In [ ]:
# The vectors
collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

# Visualizing the Vector Store

In [ ]:
# Prework 
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['doc_type'] for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange'][['model', 'motoboys', 'services', 'company'].index(t)] for t in doc_types]

In [ ]:
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show(renderer="iframe_connected")

The figure visualizes the distribution of high-dimensional document embeddings in a 2D space using t-SNE. Each point represents a document, and its color indicates its category. Points that appear closer together are likely to be semantically similar, helping us identify clusters and relationships between categories.

# LangChain

In [ ]:
# MODEL = "llama3.2"
MODEL = "phi3"

# create a new Chat with llama3.2
llm = ChatOpenAI(temperature=0.7, model_name=MODEL, base_url='http://localhost:11434/v1', api_key='ollama')

# set up the conversation memory for the chat, in case we want LLM to respond considering previous questions
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key="answer")

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# putting it together: set up the conversation chain with the MODEL, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever,return_source_documents=True, output_key="answer")

In [ ]:
%%time
# Let's try a simple question
query = "Please explain the System Flow for Demand Forecasting process in a couple of sentences"
result = conversation_chain.invoke({"question": query,"chat_history": []})
display(Markdown(result["answer"]))

In [ ]:
%%time
# Let's try a simple question
query = "Please tell me Diego Fernandes key responsabilities, in English"
result = conversation_chain.invoke({"question": query,"chat_history": []})
display(Markdown(result["answer"]))

# Feedback Saving

In [ ]:
class FeedbackEntry(BaseModel):
    timestamp: datetime = Field(default_factory=datetime.utcnow)
    user_query: str
    model_response: str
    feedback: str
    source_documents: list[str]

In [ ]:
FEEDBACK_LOG = Path("feedback_log.jsonl")

def save_feedback_async(entry: FeedbackEntry):
    logging.info("Feedback Saved")
    def _save():
        with FEEDBACK_LOG.open("a") as f:
            f.write(entry.model_dump_json() + "\n")
    Thread(target=_save).start()

In [ ]:
last_interaction = {"query": "", "answer": "", "sources": []}

def generate_response(message, history):
    result = conversation_chain({"question": message,"chat_history":[]})
    answer = result["answer"]
    sources = [doc.page_content for doc in result["source_documents"]]
    
    documents = result.get("source_documents", [])
    top_docs = documents[:2]
    doc_names = []
    for i, doc in enumerate(top_docs, 1):
        source = doc.metadata.get("source", f"Documento {i}")
        filename = source.split("/")[-1]
        doc_names.append(f"{i}. {filename}")

    rag_list = "\n".join(doc_names)

    last_interaction["query"] = message
    last_interaction["answer"] = answer
    last_interaction["sources"] = sources

    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": answer})
    return "", history,rag_list

def register_feedback(feedback_type):
    entry = FeedbackEntry(
        user_query=last_interaction["query"],
        model_response=last_interaction["answer"],
        feedback=feedback_type,
        source_documents=last_interaction["sources"]
    )
    save_feedback_async(entry)
    return f"Feedback '{feedback_type}' registered sucessfully."

# Chat Interface - Gradio

In [ ]:
# Soimplest interface
def chat(question, history):
    result = conversation_chain.invoke({"question": question,"chat_history": []})
    return result["answer"]
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=False)

In [ ]:
# More fancy user interface
with gr.Blocks() as ui:
    gr.Markdown("## 💬 LLM Deliveries Internal Chat")

    with gr.Row():
        with gr.Column(scale=4):  # Chat 
            chatbot = gr.Chatbot(type="messages")

            with gr.Row():  # Input + buttons
                msg = gr.Textbox(placeholder="Type your question...",  show_label=False, scale=9)
                with gr.Column(scale=1, min_width=60):
                    send_btn = gr.Button("➤", size="sm")

        with gr.Column(scale=1): 
            gr.Markdown("### Feedback")
            thumbs_up = gr.Button("👍 (Good Answer)")
            thumbs_down = gr.Button("👎 (Bad Answer)")
            feedback_status = gr.Textbox(label="Feedback Status")

            gr.Markdown("### Relevant Documents")
            rag_docs = gr.Textbox(lines=6, interactive=False, label=None)

    send_btn.click(generate_response, inputs=[msg, chatbot], outputs=[msg, chatbot, rag_docs])
    msg.submit(generate_response, inputs=[msg, chatbot], outputs=[msg, chatbot, rag_docs])
    thumbs_up.click(fn=lambda: register_feedback("thumbs_up"), outputs=feedback_status)
    thumbs_down.click(fn=lambda: register_feedback("thumbs_down"), outputs=feedback_status)


In [ ]:
ui.launch()

# Metrics

In [ ]:
import pandas as pd

In [ ]:
# reading feedbacks
df = pd.read_json(Path("feedback_log.jsonl"), lines=True)

In [ ]:
# Calculating percentage of correct answers
df["feedback_binario"] = df["feedback"].map({"thumbs_up": 1, "thumbs_down": 0})
correct_answer_rate = df["feedback_binario"].mean() * 100